In [60]:
import pandas as pd
import json

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [100]:
supplements = pd.read_csv("Supplements.csv")
supplements.head(5)

,Name,Url
0,Abscess Root,https://www.webmd.com/vitamins/ai/ingredientmo...
1,Abuta,https://www.webmd.com/vitamins/ai/ingredientmo...
2,Acacia Rigidula,https://www.webmd.com/vitamins/ai/ingredientmo...
3,Acai,https://www.webmd.com/vitamins/ai/ingredientmo...
4,Acerola,https://www.webmd.com/vitamins/ai/ingredientmo...


In [101]:
supplements = supplements[supplements["Name"].str.startswith('A')]
supplements.tail(5)

,Name,Url
100,Avens,https://www.webmd.com/vitamins/ai/ingredientmo...
101,Avocado,https://www.webmd.com/vitamins/ai/ingredientmo...
102,Avocado Soy Unsaponifiables (Asu),https://www.webmd.com/vitamins/ai/ingredientmo...
103,Avocado Sugar Extract,https://www.webmd.com/vitamins/ai/ingredientmo...
104,Ayahuasca,https://www.webmd.com/vitamins/ai/ingredientmo...


In [92]:
def get_element_text(supplementName, elementSection, elementPath):

    try:
        element = wait_time.until(EC.presence_of_element_located((By.XPATH, elementPath)))
        text = element.text.strip()
    except Exception as e:
        text = f"No {elementSection} information for {supplementName}"
    finally:
        return text

In [93]:
def get_usess(elementPath):

    uses = {}

    try:
        element = wait_time.until(EC.presence_of_element_located((By.XPATH, elementPath)))
        h3_tags = element.find_elements(By.TAG_NAME, "h3")
        
        ul_counter = 1
        for h3 in h3_tags:
            uses_per_tag = []

            ul_path = f"/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]/div[{ul_counter}]/ul"
            ul_element = element.find_elements(By.XPATH, ul_path)

            li_elements = ul_element.find_elements(By.TAG_NAME, "li")
            for item in li_elements:
                li_text = item.strip()
                uses_per_tag.append(li_text)

            uses[h3.text] = uses_per_tag
            ul_counter += 1

        #print(json.dumps(uses, indent=4))
        print(uses)
    except Exception as e:
        print(e)

In [94]:
def get_uses(elementPath):
    uses = {}

    try:
        element = wait_time.until(EC.presence_of_element_located((By.XPATH, elementPath)))
        h3_tags = element.find_elements(By.TAG_NAME, "h3")
        
        ul_counter = 1
        for h3 in h3_tags:
            uses_per_tag = []

            ul_path = f"/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]/div[{ul_counter}]/ul"
            ul_elements = element.find_elements(By.XPATH, ul_path)

            if ul_elements:
                li_elements = ul_elements[0].find_elements(By.TAG_NAME, "li")
                
                for item in li_elements:
                    li_text = item.text.strip() 
                    uses_per_tag.append([li_text])

                uses[h3.text] = uses_per_tag
            ul_counter += 1

    except Exception as e:
        print(e)
    finally:
        return uses

In [102]:
supplements

,Name,Url
0,Abscess Root,https://www.webmd.com/vitamins/ai/ingredientmo...
1,Abuta,https://www.webmd.com/vitamins/ai/ingredientmo...
2,Acacia Rigidula,https://www.webmd.com/vitamins/ai/ingredientmo...
3,Acai,https://www.webmd.com/vitamins/ai/ingredientmo...
4,Acerola,https://www.webmd.com/vitamins/ai/ingredientmo...
...,...,...
100,Avens,https://www.webmd.com/vitamins/ai/ingredientmo...
101,Avocado,https://www.webmd.com/vitamins/ai/ingredientmo...
102,Avocado Soy Unsaponifiables (Asu),https://www.webmd.com/vitamins/ai/ingredientmo...
103,Avocado Sugar Extract,https://www.webmd.com/vitamins/ai/ingredientmo...


In [103]:
supplement_info = {}

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for _, info in supplements.head(2).iterrows():

    supplementName = info.Name
    supplementURL = info.Url

    driver.get(supplementURL)
    wait_time = WebDriverWait(driver,5)
    overview = get_element_text(supplementName, "Overview", "/html/body/div[1]/div/main/div/div[2]/div/div/div[1]/div[1]/div")
    uses = get_uses("/html/body/div[1]/div/main/div/div[2]/div/div/div[2]/div[1]")
    sideEffects = get_element_text(supplementName, "Side Effects", "/html/body/div[1]/div/main/div/div[2]/div/div/div[3]/div[1]")
    precautions = get_element_text(supplementName, "Precautions", "/html/body/div[1]/div/main/div/div[2]/div/div/div[4]/div[1]/div")
    interactons = []
    dosing = get_element_text(supplementName, "Dosing", "/html/body/div[1]/div/main/div/div[2]/div/div/div[6]/div[1]/div")

    supplement_info[supplementName] = {
        "Overview": overview,
        "Uses": uses,
        "SideEffects": sideEffects,
        "Precautions": precautions,
        "Interactions": interactons,
        "Dosing": dosing 
    }

driver.quit()

In [104]:
print(json.dumps(supplement_info, indent=4))

{
    "Abscess Root": {
        "Overview": "Abscess root (Polemonium reptans) is an herb. The root is ground and used as a medicinal tea, but it's not clear if it has any benefits.\n\nAbscess root is used for fever, cough, swelling, and other conditions. But there is no good scientific evidence to support any uses.\n\nAbscess root is also called \"false Jacob's ladder,\" because it has some of the same effects as another plant called Jacob's ladder. These two plants are used in similar ways, but are not the same.",
        "Uses": {},
        "SideEffects": "When taken by mouth: There isn't enough reliable information to know if abscess root is safe. It might cause side effects such as stomach upset and sneezing.",
        "Precautions": "No Precautions information for Abscess Root",
        "Interactions": [],
        "Dosing": "Ground abscess root is traditionally used to make a tea, but there isn't enough reliable information to know what an appropriate dose might be. Keep in mind 